In [1]:
import boto3
import os
from dotenv import load_dotenv

load_dotenv()
access_key_id = os.getenv("ACCESS_KEY_ID")
secret_access_key = os.getenv("SECRET_ACCESS_KEY")
minio_url = "http://" + os.getenv("S3_API_ENDPOINT")


minio_client = boto3.client(
    "s3",
    aws_access_key_id=access_key_id,
    aws_secret_access_key=secret_access_key,
    endpoint_url=minio_url
)

new_bucket = "trusted-zone"
try:
    minio_client.create_bucket(Bucket=new_bucket)
except (minio_client.exceptions.BucketAlreadyExists, minio_client.exceptions.BucketAlreadyOwnedByYou):
    print(f"Bucket '{new_bucket}' already exists")

Bucket 'trusted-zone' already exists


In [2]:
from PIL import Image, ImageEnhance, ImageFilter
import io
from tqdm import tqdm

bucket_origen = "persistent-landing"
bucket_desti = "formatted-zone"
prefix_origen = "images/"
mida_final = (600, 450)

paginator = minio_client.get_paginator("list_objects_v2")

for page in paginator.paginate(Bucket=bucket_origen, Prefix=prefix_origen):
    for obj in tqdm(page.get("Contents", []), desc="Processant imatges"):
        key = obj["Key"]
        filename = key.split("/")[-1]

        #Llegir la imatge des del bucket
        response = minio_client.get_object(Bucket=bucket_origen, Key=key)
        image_data = response["Body"].read()
        #Obrir amb PIL per validar i redimensionar
        try:
            img = Image.open(io.BytesIO(image_data)).convert("RGB")
        except Exception as e:
            print(f"Error amb {filename}: {e}")
            continue

        #Redimensionar a mida consistent
        img = img.resize(mida_final)
        
        # APLICAR TRATAMIENTOS DE MEJORA
        
        # 1. Normalización de colores (ajustar brillo y contraste)
        # Ajustar brillo (ligeramente más brillante)
        brightness_enhancer = ImageEnhance.Brightness(img)
        img = brightness_enhancer.enhance(1.1)  # +10% brillo
        
        # Ajustar contraste (mejorar contraste)
        contrast_enhancer = ImageEnhance.Contrast(img)
        img = contrast_enhancer.enhance(1.15)  # +15% contraste
        
        # Ajustar saturación (colores más vivos)
        color_enhancer = ImageEnhance.Color(img)
        img = color_enhancer.enhance(1.05)  # +5% saturación
        
        # 2. Suavizado para reducir ruido
        img = img.filter(ImageFilter.GaussianBlur(radius=0.3))  # Suavizado ligero
        
        # 3. Aplicar filtro de nitidez para compensar el suavizado
        img = img.filter(ImageFilter.SHARPEN)

        #Desa a memòria i puja a trusted-zone
        buffer = io.BytesIO()
        img.save(buffer, format="PNG", optimize=True)
        buffer.seek(0)

        new_key = f"images/{filename}"
        minio_client.put_object(
            Bucket=bucket_desti,
            Key=new_key,
            Body=buffer
        )

Processant imatges: 100%|██████████| 100/100 [00:19<00:00,  5.15it/s]
